# Crawler

In [1]:
pip install scrapy

     |████████████████████████████████| 245kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 245kB 45.6MB/s 
     |████████████████████████████████| 3.1MB 46.6MB/s 
     |████████████████████████████████| 2.6MB 30.2MB/s 
     |████████████████████████████████| 3.2MB 36.6MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=aeb3cc51d6974b1e3d0077b92c62fe6bb514a8c0b1ab0073d9db822927fe9895
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=0fe05f0800d5a496bedaecd671a871835fb7f4bfc8d9d7ef905979982732a437
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatch

In [83]:
########## Module import ##########

# Files
from google.colab import files
import json
import pandas as pd

# Maths
import random

# Extraction
import re

# Scraping
import scrapy
#from scrapy import Selector
from requests import get

# Parsing
from urllib.parse import urlencode
from urllib.parse import urlparse

# Format
from datetime import datetime
from datetime import date
from datetime import timedelta
import time

In [3]:
# Upload files
uploaded = files.upload()

Saving Lexique_Gammes_Gestion.txt to Lexique_Gammes_Gestion.txt
Saving Lexique_Innovation.txt to Lexique_Innovation.txt
Saving listCouple.json to listCouple.json


In [65]:
# Get the list of word combinations
df = pd.read_json('listCouple.json', orient='index')
df_t = df.T
listCouple=df_t.values.tolist()
print(listCouple)

[['"crédit de paiement"', '"brain machine interface"'], ['"gestion budgétaire"', '"chatbot"'], ['"évaluation"', '"robotique sociale"'], ['"Finance"', '"infovis"'], ['"pilotage de l\'achat"', '"université 4.0"'], ['"Paie"', '"du futur"'], ['"management opérationnel"', '"big-data"'], ['"relations sociales"', '"nlp"'], ['"Gestion financière"', '"augmented intelligence"'], ['"Mairie"', '"innovation"'], ['"SIRH"', '"si"'], ['"élus"', '"infovis"'], ['"carrière"', '"visualisation d\'informations"'], ['"gestion de la comptabilité"', '"microservices"'], ['"GPEC"', '"systèmes d\'information"'], ['"règlement de consultation"', '"système de recommandation"'], ['"compétences"', '"bmi"'], ['"gestion électronique des documents"', '"traitement automatique du langage"'], ['"immobilisation"', '"graphes de connaissances"'], ['"talents"', '"design thinking"'], ['"anomalie"', '"mr"'], ['"retraites"', '"véhicule autonome"'], ['"crédit de paiement"', '"université du futur"'], ['"retraites"', '"material desig

In [66]:
########## Parameters to change ##########

# API Key (created on Scraper API)
API_KEY = 'fff2df1787bf81bfe98277920f9a9fcb'

# Test on a few couples
p_listCouple = listCouple[0:49]
# Number of couples
p_length = 1
# Number of requests
p_requestNumber = 2

In [7]:
#opening key word.txt and turning them into lists
# with open('Lexique_Gammes_Gestion.txt') as img:
#     gestion = img.readlines()
# with open('Lexique_Innovation.txt') as img:
#     innovation = img.readlines()
# for i in range(len(gestion)):
#   gestion[i]=gestion[i][:-1]
# print(gestion)
# for i in range(len(innovation)):
#   innovation[i]=innovation[i][:-1]
# print(innovation)

In [8]:
# def addGuillemet(listMot):
#   newList=[]
#   for mot in listMot :
#     newList.append('"'+mot+'"')
#   return newList
# gestion=addGuillemet(gestion)
# innovation=addGuillemet(innovation)

In [9]:
#creating a list by making the cartesian product of innovation.txt and gestion.txt
# listCouple : list=[] #the result
# for motG in gestion :
#   for motI in innovation :
#     listCouple.append([motG,motI])
# #tests
# print(listCouple)
# print(len(listCouple))

In [86]:
def create_google_url(query):
    """ 
    Allows you to create a Google URL from a keyword

    Parameter :
        query : keyword to enter in the search bar
    
    Out :
        google_url : google URL created from the keyword
    """
    # num = number of results to be scraped
    google_dict = {'q': query, 'num': 200,}
    google_url = 'http://www.google.com/search?' + urlencode(google_dict)
    return google_url

def combAND(couple):
    """Documentation
    Parameters:
        couple: a list of 2 Strings
    Out :
        list : a combination of the 2 members of a couple with AND between them
    """

    return str(couple[0])+' '+'AND'+' '+str(couple[1])

def listToAND(listCouple):
    """Documentation
    Parameters:
        listCouple: a list of couple

    Out :
        list : a list of the combination of the couple of listCouple
    """

    #we use combAND
    return [combAND(i) for i in listCouple]


def combOR(tuple):
    """Documentation

    Parameters:
        tuple: a list of String

    Out :
        final : a combination of the members of the tuple with OR between them and framed with ()
    """

    #first step : initialisation of final
    final='('+str(tuple[0])+')'
    #second step : adding the rest of the tuple
    for i in range(1,len(tuple)):
        final=final+'|'+'('+tuple[i]+')'
    return final

#applying the previous function to a list of tuple
def listToOR(listTuple):
    """Documentation
    Parameters:
        listTuple: a list of tuples

    Out :
        list : a list of the combination of the tuple of listTuple
    """

    #we use combOR
    return [combOR(i) for i in listTuple]

#making a list of random tuples
def listComb(listAND,numbT=2,iteration=int(1000)): #we have to limit the number of request, by default 1000, and we make couples (2)
    """Documentation

    Parameters:
        listAND: a list Strings with AND
        numbT : the length of the tuple we want to create
        iteration : the maximum number of combination we want to create

    Out :
        finalList : a list of the combination of the tuple of listTuple
    """

    finalList=[]
    i=0
    #Step 1 : we loop until we have enough tuples or the list is empty
    while ((len(listAND)>=numbT) and (i<iteration)):
        i+=1
        #Step 2 : at each loop, we take some random elements of listAND and create a tuple with them
        listRand=random.sample(listAND,numbT)
        #Step 3 : we remove the elements from listAND
        for j in listRand :
            listAND.remove(j)
        #Step 4 :we add the tuple we created to our finalList
        finalList.append(listRand)
    return finalList


def link_sub_month(link, nbmonths):
    """Documentation
    Function that allows to add a "limit date" parameter in the link

    Parameters:
        link : URL to which the date filter should be added
        nbmonths : Limit date (x months before today)

    Out :
        link_new : URL with a date filter
    """

    today = date.today()    
    nbdays=nbmonths*30
    days_to_substract = timedelta(days=nbdays)
    date_limit = today - days_to_substract
    # Limit date    
    jour=str(date_limit.day)
    mois=str(date_limit.month)
    annee=str(date_limit.year)
    
    link_new = link+"&source=lnt&tbs=cdr%3A1%2Ccd_min%3A"+mois+"%2F"+jour+"%2F"+annee+"%2Ccd_max%3A&tbm="
    return link_new

def get_url(url):
    """ 
    Creation of the URL that will allow the legal scraping of Google results (use of the API key). 
    This URL is equivalent to a Google search.

    Parameter :
        url : google URL created from the keyword
    
    Out :
        proxy_url : URLs built using the API
    """

    payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr', 'pws': 0}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    date_url = proxy_url + link_sub_month(proxy_url, 6)
    return proxy_url

class GoogleSpider(scrapy.Spider):
    """ 
    This class lists functions for scraping Google results from a list of keywords
    """

    # GoogleSpider class name
    name = 'google'
    # Name of the site to be scraped
    allowed_domains = ['www.google.com']
    # Settings
    custom_settings = {
                        # Criticality level at which the log is displayed
                        'LOG_LEVEL': 'INFO', 
                        # Maximum number of simultaneous requests 
                        'CONCURRENT_REQUESTS_PER_DOMAIN': 1, 
                        # Maximum number of retries to be made if the query fails
                        'RETRY_TIMES': 0}

    def start_requests(self, listCouple, length, requestNumber):
        #Initialisation of DataFrame
        df = pd.DataFrame(columns=['URL','Query'])
        #Adding quotes
        #listCouple = listExpression(listCouple)
        #Format changeover 
        lWork=listToAND(listCouple)
        #Selection of queries 
        lWork=listComb(lWork,numbT=length, iteration=requestNumber)
        #We change the format of Keywords
        lWork=listToOR(lWork)

        lURL=[]
        #We loop the keywords to generate the queries 
        for query in lWork:
            url = create_google_url(query)
            lURL.append(str(scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0}))[5:-1])
        #column generation
        df['Query'] = lWork
        df['URL'] = lURL

        yield df

    def parse(self, response):
        di = json.loads(response.text)
        pos = response.meta['pos']
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        for result in di['organic_results']:
            title = result['title']
            snippet = result['snippet']
            link = result['link']
            item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
            pos += 1
            yield item
        next_page = di['pagination']['nextPageUrl']
        if next_page:
            yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [87]:
# Start building URLs
if __name__ == '__main__':
    df_result = list(GoogleSpider().start_requests(listCouple=p_listCouple,length=p_length,requestNumber=p_requestNumber))[0]

In [88]:
# Crawling start time
print(datetime.now().strftime("%H:%M:%S"))

15:28:10


In [89]:
list_source = []
i = 0

for index, row in df_result.iterrows():
    link = row['URL']
    query = row['Query']
    # 1 minute break to avoid API overloading
    time.sleep(60)
    # URL scraping
    response = get(link)

    # Test if the request was successful
    if response.status_code == 200:
        # Addition of the scraped google results and the corresponding query
        list_source.append([response.text, query])

        i+=1
        # Saving the results every 20 queries
        if (i%20==0):
            with open('etape'+str(i)+'.json', 'w') as jsonfile:
                json.dump(list_source, jsonfile)
            files.download('etape'+str(i)+'.json')


In [90]:
# Crawling end time
print(datetime.now().strftime("%H:%M:%S"))

15:30:37


In [60]:
my_links = []
# Pattern for extracting URLs
pattern = re.compile("\"link\"[^,]+")

df_sources = pd.DataFrame(columns=['URL','Query'])
for source in list_source:

    # Finding patterns in the scraped results
    clean = pattern.findall(str(source[0]))
    # Sorting the results
    clean_order = dict.fromkeys(clean)
    my_links = list(dict.fromkeys([x for x in clean_order]))
    # Extraction of URLs
    my_links = [i[8:] for i in my_links]
    my_links = [i[:-1] for i in my_links]

    for i in my_links:
        # Check that the URL is a result (different from Google)
        if "https://www.google" not in i:
            # For each URL, keep the query that led to finding it
            df_sources = df_sources.append({'URL':i, 'Query':source[1]}, ignore_index=True)
    
df_sources

,URL,Query
0,https://www.gcu.edu/blog/engineering-technolog...,"(""GED"" AND ""edge ai"")"
1,https://www.globenewswire.com/news-release/202...,"(""GED"" AND ""edge ai"")"
2,https://www.globenewswire.com/news-release/202...,"(""GED"" AND ""edge ai"")"
3,http://www.dernieresactus.fr/maru-launches-ins...,"(""GED"" AND ""edge ai"")"
4,https://www.researchgate.net/publication/34025...,"(""GED"" AND ""edge ai"")"
5,https://ec.europa.eu/futurium/en/system/files/...,"(""GED"" AND ""edge ai"")"
6,https://marumatchbox.com/press_release/maru-ma...,"(""GED"" AND ""edge ai"")"
7,https://www.aijobs.tech/,"(""GED"" AND ""edge ai"")"
8,https://itrnews.com/articles/187026/docuware-u...,"(""GED"" AND ""edge ai"")"
9,https://infodsi.com/articles/183530/documation...,"(""GED"" AND ""edge ai"")"


In [61]:
len(df_sources)

39

In [31]:
# Storing the data in JSON format
df_sources.to_json('liens_crawler.json', orient = 'split')

In [32]:
# Readind the JSON file 
# df = pd.read_json('liens_crawler.json', orient ='split') 

In [ ]:
# Download the JSON file
files.download('liens_crawler.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>